# RL Exploration considering Uncertainty

# 1. Theoretical view

## 1-1. Posterior Sampling for Reinforcement Learning (PSRL)

Assume the reward $\mu$ and the transition $P$ is stochastic. For each episode, one sample an MDP from the posterior distribution, conditioned on the history $\mathcal{F}_{t}$ that is generated up to episode $t$. Then, the algorithm computes the optimal policy given the sampled MDP.

## 1-2. Uncertainty Bellman Equation (UBE)

Assume the posterior distributions of $\mu$ and $P$ can be derived. Then, $\textbf{Uncertainty Bellman Equation}$ is

\begin{equation}
u_{sa}^{h}
=\nu_{sa}^{h}+\sum_{s'a'}\pi_{s'a'}^{h}\mathbb{E}\left( P_{s'sa}^{h} | \mathcal{F}_{t} \right)u_{s'a'}^{h+1}
\end{equation}

where $\nu_{sa}^{h}$ is the local uncertainty at $(s,a)$, which is given by

\begin{equation}
\nu_{sa}^{h}=\mathrm{Var}\left(\hat{\mu}_{sa}^{h} | \mathcal{F}_{t} \right)+Q_{\mathrm{max}}^{2}\sum_{s'}\frac{\mathrm{Var}\left(\hat{P}_{s'sa}^{h} | \mathcal{F}_{t} \right)}{\mathbb{E}\left( \hat{P}_{s'sa}^{h} | \mathcal{F}_{t} \right)}
\end{equation}

Given UBE, one can approximate the posterior distribution of Q value as

\begin{gather*}
Q_{sa}^{h}\sim\mathcal{N}\left( \bar{Q}_{sa}^{h}, \mathrm{diag}(u_{sa}^{h}) \right) \\
\bar{Q}_{sa}^{h}=\mathbb{E}\left( \hat{\mu}_{sa}^{h} | \mathcal{F}_{t} \right)+\sum_{s',a'}\pi_{s'a'}^{h}\mathbb{E}\left( \hat{P}_{s'sa}^{h} | \mathcal{F}_{t} \right)\bar{Q}_{s'a'}^{h+1}
\end{gather*}

and use it to perform Thompson sampling(=posterior sampling) as an exploration strategy.

\begin{gather*}
a=\mathrm{argmax}_{b}\left( \bar{Q}_{sb}^{h}+\epsilon_{b}\cdot\left( u_{sb}^{h} \right)^{0.5} \right) \\
\epsilon_{b}\sim\mathcal{N}(0,1)
\end{gather*}

# 2. Model-free Reinforcement Learning

## 2-1. Bootstrapped DQN

Bootstrapped DQN uses multi-head Q function. For each episode, the agent sample a head randomly and follow greedy policy w.r.t. the sampled head.

## 2-2. UCB Exploration via Q-Ensembles

Q-ensemble agent also uses multi-head Q function. However, Q-ensemble agent does not sample a head. Instead, the agent uses whole the heads to calculate the value function. 

In addition, it adapts an exploration strategy which is based on 'optimism in the face of uncertainty (OFU)'. By adding a empirical standard deviation to the average of Q ensemble heads, and choosing the action whose optimistic Q estimate is highest, the agent deals with the problem of exploration-exploitation dilemma.

## 2-3 Bayes-by-Backprop Q-network

# 3. Model-based Reinforcement Learning

## 3-1. Probabilistic Inference for Learning Control (PILCO)

Assume deterministic transition model with the Gaussian noise, and the objective is to find a policy $\pi$ that minimize the expected total cost (how far away a state is from the destination)

\begin{align*}
\text{Transition: }s_{t+1} &= f(s_{t},a_{t}) \\
\text{Target: }\Delta_{t} &= s_{t+1} - s_{t} + \varepsilon \\
\end{align*}
\begin{equation}
\varepsilon \sim \mathcal{N}(0,\mathrm{diag}(\sigma_{\varepsilon}))
\end{equation}

\begin{gather*}
\mathrm{Obj: }\min_{\theta}\mathcal{J}^{\pi}(\theta)=\sum_{t=0}^{T}\mathbb{E}_{s_{t}}[ c(s_{t}) ] \\
c(s_{t}) = 1-\exp\left( -\frac{1}{2\sigma_{c}^{2}}\|s_{t}-s_{\mathrm{target}}\|^{2} \right)
\end{gather*}

In order to consider model uncertainty, which is essential for the prediction of transitions to unknown states, probabilistic model is proposed. In PILCO, Gaussian Process is adapted as a dynamics model.

- One-step prediction model

\begin{align*}
p(s_{t+1} | s_{t},a_{t})
&= \mathcal{N}\left( s_{t+1} | \mu_{t+1}, \Sigma_{t+1} \right) \\
\mu_{t+1} &=  s_{t} + \mathbb{E}_{f}[ \Delta_{t} ] \\
\Sigma_{t+1} &= \mathrm{Var}_{f}[ \Delta_{t} ]
\end{align*}

- GP dyamics model

\begin{gather*}
\mathbf{x}:=(s,a) \\
k(\mathbf{x}_{t},\mathbf{x}_{t'}) = \alpha^{2}\exp \left(-\frac{1}{2}||\mathbf{x}_{t}-\mathbf{x}_{t'}||_{\mathbf{\Lambda}}\right)
\end{gather*}

\begin{equation}
f \sim GP(m_{f}, k(\cdot,\cdot))
\end{equation}

Evaluating $\mathcal{J}^{\pi}$ requires the state distributions $p(s_{1}), ... , p(s_{T})$, which would be approximated by using one-step prediction and GP dynamics model. Then, policy evaluation $\mathcal{J}^{\pi}(\theta)$ and policy gradient $\mathrm{d}\mathcal{J}^{\pi}(\theta)/\mathrm{d}\theta$ can be computed analytically.

However, it does not scale well w.r.t. bthe number of trials and the dimension of state and action. Also, it cannot consider the temporal correlation of uncertainty in successive steps. DeepPILCO tried to handle these limitations by using Bayesian deep dynamics model and particle method.

- One-step prediction model of DeepPILCO

\begin{equation}
p(s_{t+1} | s_{t},a_{t}) = \mathrm{MCDropout}(s_{t}, a_{t};\mathbf{w}, p)
\end{equation}

- Bayesian deep dynamics model

\begin{gather*}
\mathbf{w} \sim p(\mathbf{w} | \mathcal{D}) \\
f_{\mathbf{w}} = \mathrm{BayesRNN}(\mathbf{w})
\end{gather*}

- Input distribution estimation

\begin{align*}
s_{t}^{(k)} &\sim p(s_{t}),\; a_{t}^{(k)} \sim \pi (a_{t} | s_{t}^{(k)}) \\
s_{t+1}^{(k)} &= f_{\mathbf{w}}(s_{t}^{(k)},a_{t}^{(k)}) \\
\hat{\mu}_{t+1} &= \frac{1}{K}\sum_{k=1}^{K}s_{t+1}^{(k)} \\
\hat{\sigma}_{t+1}^{2} &= \frac{1}{K}\sum_{k=1}^{K}\left( s_{t+1}^{(k)} - \mu_{t+1} \right)^{2} \\
p(s_{t+1}) &\approx \mathcal{N}(\hat{\mu}_{t+1}, \hat{\sigma}_{t+1}^{2})
\end{align*}

## 3-2. Probabilistic Ensembles with Trajectory Sampling

## 3-3. Model-Ensemble TRPO